In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf

In [2]:
d1=pd.read_csv("sql12.csv")
d2=pd.read_csv("apache-ant-1.7.0.csv")
d3=pd.read_csv("apache-jmeter-2.10.csv")
d4=pd.read_csv("argouml.csv")
d5=pd.read_csv("columba-1.4-src.csv")
d6=pd.read_csv("emf-2.4.1.csv")
d7=pd.read_csv("hibernate-distribution-3.3.2.GA.csv")
d8=pd.read_csv("jEdit-4.2.csv")
d9=pd.read_csv("jruby-1.4.0.csv")

In [3]:
frames=[d1,d2,d3,d4,d5,d6,d7,d8,d9]
data=pd.concat(frames)
data.head(3)

,projectname,classification,commenttext
0,sql12,DEFECT,/**\r\t * TODO: this is a bug in the code at t...
1,sql12,DEFECT,// Cast to int required as Dimension::setSize(...
2,sql12,DEFECT,/*\r * This won't work on Derby where no...


In [4]:
found = data[data['classification'].str.contains('WITHOUT_CLASSIFICATION')]
print(found.count())

projectname       54005
classification    54005
commenttext       54005
dtype: int64


In [5]:
def form_label(row):
    if row['classification'] == 'WITHOUT_CLASSIFICATION':
        return 'no'
    else: return 'yes'
data.apply (lambda row: form_label(row), axis=1)

0       yes
1       yes
2       yes
3       yes
4       yes
       ... 
4892     no
4893     no
4894     no
4895     no
4896     no
Length: 57867, dtype: object

In [6]:
data['label'] = data.apply (lambda row: form_label(row), axis=1)
data.shape

(57867, 4)

In [7]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 57867 entries, 0 to 4896
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   projectname     57867 non-null  object
 1   classification  57867 non-null  object
 2   commenttext     57867 non-null  object
 3   label           57867 non-null  object
dtypes: object(4)
memory usage: 2.2+ MB


In [8]:
data.head(3)

,projectname,classification,commenttext,label
0,sql12,DEFECT,/**\r\t * TODO: this is a bug in the code at t...,yes
1,sql12,DEFECT,// Cast to int required as Dimension::setSize(...,yes
2,sql12,DEFECT,/*\r * This won't work on Derby where no...,yes


In [57]:
from collections import Counter

def get_class_weights(y):
    counter = Counter(y)
    #majority = max(counter.values())
    #return  {cls: round(float(majority)/float(count), 2) for cls, count in counter.items()}
    cw= [(i, counter[i] / len(y)) for i in counter]
    
    classweight =[cw[1][1]]
    classweight.append(cw[0][1])
    print(cw)
    print(classweight)
    return classweight

class_weights= get_class_weights(data.label.values) #returns minority class weight
#print(class_weights)


[('yes', 0.06673924689373908), ('no', 0.9332607531062609)]
[0.9332607531062609, 0.06673924689373908]


In [102]:
#Binary Cross-Entropy: y * log(p) + (1-y) * log(1-p)
'''

from math import log
from numpy import mean
 
# calculate cross entropy
def cross_entropy(y_true, y_pred):
    return -sum([y_true[i]*log(y_pred[i]) for i in range(len(y_true))])
 
# define classification data
ytrue = [1, 1, 0, 0, 0, 0, 0, 0, 0, 0]
ypred = [0.8, 0.9, 0.5, 0.4, 0.8, 0.1, 0.4, 0.2, 0.1, 0.3]
# calculate cross entropy for each example
results = list()
for i in range(len(ytrue)):
    # create the distribution for each event {0, 1}
    expected = [1.0 - ytrue[i], ypred[i]]
    predicted = [1.0 - ypred[i], ypred[i]]
    print(expected)
    # calculate cross entropy for the two events
    ce = cross_entropy(expected, predicted)
    print('>[y=%.1f, yhat=%.1f] ce: %.3f nats' % (ytrue[i], ypred[i], ce))
    results.append(ce)
 '''



"\n\nfrom math import log\nfrom numpy import mean\n \n# calculate cross entropy\ndef cross_entropy(y_true, y_pred):\n    return -sum([y_true[i]*log(y_pred[i]) for i in range(len(y_true))])\n \n# define classification data\nytrue = [1, 1, 0, 0, 0, 0, 0, 0, 0, 0]\nypred = [0.8, 0.9, 0.5, 0.4, 0.8, 0.1, 0.4, 0.2, 0.1, 0.3]\n# calculate cross entropy for each example\nresults = list()\nfor i in range(len(ytrue)):\n    # create the distribution for each event {0, 1}\n    expected = [1.0 - ytrue[i], ypred[i]]\n    predicted = [1.0 - ypred[i], ypred[i]]\n    print(expected)\n    # calculate cross entropy for the two events\n    ce = cross_entropy(expected, predicted)\n    print('>[y=%.1f, yhat=%.1f] ce: %.3f nats' % (ytrue[i], ypred[i], ce))\n    results.append(ce)\n "

In [59]:
mean_ce = mean(results)
print('Average Cross Entropy: %.3f nats' % mean_ce)

Average Cross Entropy: 0.679 nats


In [89]:
from tensorflow.keras import backend as K
def cus_loss(y_true, y_pred):
    eps= K.epsilon()
    loss = -(y_true*tf.math.log(y_pred + eps) + (1-y_true)*tf.math.log(y_pred + eps))
    return mean(loss)

In [90]:
y_true = np.array([[0, 1], [0, 0], [0,1]])
y_pred =np.array([[0.6, 0.4], [0.4, 0.6], [0.8, 0.2]])
l=cus_loss(y_true, y_pred)
l

0.781135452782582

In [56]:
tf.keras.losses.binary_crossentropy(y_true, y_pred)

<tf.Tensor: shape=(3,), dtype=float64, numpy=array([0.91629048, 0.71355797, 1.60943741])>

In [99]:
#Using in built
y_true = [[0., 1.], [0., 0.]]
y_pred = [[0.6, 0.4], [0.4, 0.6]]
# Using 'auto'/'sum_over_batch_size' reduction type.
bce = tf.keras.losses.BinaryCrossentropy()
bce(y_true, y_pred).numpy()


0.81492424

In [101]:
loss= bce(y_true, y_pred, sample_weight=get_class_weights(data.label.values)).numpy()
loss

[('yes', 0.06673924689373908), ('no', 0.9332607531062609)]
[0.9332607531062609, 0.06673924689373908]


0.45138013